# Literate computing should be permissive.

When building an idea, natural language typically proceeds code using Markdown and Code, respectively.  Currently, notebook composition decouples
these traits; in [a previous document](http://nbviewer.jupyter.org/github/deathbeds/deathbeds.github.io/blob/master/deathbeds/2018-09-02-SyntaxError-fallback.ipynb) we outlined the differences and similarities between the two.

A literate document desires a tight connection between natural language and code.  As authors, we are still in the
early stages of learning to write about code in common conversations. 

In this notebook, we try to mitigate `SyntaxError` by allowing prose in code cells.  The code cells, if they are not valid source code, defer to a Markdown transformer that executes the inline and block code in the Markdown.  

A computer may not execute code in a Markdown cell, but a consumer may replicate or reuse that code.  __Anything that is code should compute__.  Authoring markdown in code cells allows an author to validate the code in their prose.

> A benefit of composing literate particles in code cells is that an author may use tab completion and inspection.

In [9]:
from deathbeds.__Markdown_code_cells import CallableTransformer, CodeRenderer
from textwrap import dedent; import fnmatch, nbconvert; from nbformat import v4
import IPython, sys, traceback
from contextlib import contextmanager
from toolz.curried import *
from deathbeds.__Custom_display_formatting import triggers
ip = get_ipython()

The first errors that could be encountered in `IPython` when interacting with code are `SyntaxError` & 
`IndentationError`; `assert issubclass(IndentationError, SyntaxError)`.  These `Exceptions` deny any code from being compiled and executing.  

There are a few `SyntaxError` `Exception`s

In [10]:
[x for x in vars(__import__('builtins')).values() if isinstance(x, type) and issubclass(x, SyntaxError)]

[SyntaxError, IndentationError, TabError]

In our problem, we have to create some `Exception`s that are `SyntaxError`s.  [They must base class `SyntaxError` because we will using the `ip.input_transformer_manager`](https://ipython.readthedocs.io/en/stable/config/inputtransforms.html) & any other `Exception` will unregister the transformer.

__1.__ `InlineCodeCellMagic` - An author created a cell magic with inline code.  The two don't mix well.

In [4]:
class InlineCodeCellMagic(SyntaxError): ...

__2.__ `NoMarkdownCodeException` -An author created a cell magic with inline code.  The two don't mix well.

## The Input Transformer.

Our `Literate` renderer will extract inline and block code as code input.

In [5]:
    class Literate(CallableTransformer, CodeRenderer):
        def code(self, node, entering): self.nodes.append(node)

        def __call__(self, str):
            self.nodes = []
            current, lines, str = 0, str.splitlines(), CodeRenderer.__call__(self, str)
            error = None
            
            if str.lstrip().startswith('%%') and self.nodes: 
                raise InlineCodeCellMagicException("""Inline code may not be mixed with cell magics.""") from Exception
            
            if lines and not(self.nodes or str.strip()):
                raise NoMarkdownCodeException("No code objects discoved in the Markdown code.") from Exception
            
            str = dedent(str or '\n').splitlines()
            
            while """Increment the lines until there are enough of them.
            """ and (len(str) < len(lines)): str.append("")

            for node in self.nodes:
                for current in range(current, len(lines)):
                    try:
                        id = lines[current].index(f"`{node.literal}`")
                        break
                    except ValueError: ...
                str[current] += ('; ' if str[current].strip() else '') + node.literal
                
            return '\n'.join(str)

`swap_input_transformer` is a `contextmanager` that temporaily attaches `Literate` to the transformers
on the 🥇 `SyntaxError` using the `ip.input_transformer_manager.physical_line_transforms`.

In [6]:
@contextmanager
def swap_input_transformer(object, position=0):
    ip = get_ipython()
    yield ip.input_transformer_manager.physical_line_transforms.insert(position, object) or ip
    ip.input_transformer_manager.physical_line_transforms = [
        x for x in get_ipython().input_transformer_manager.physical_line_transforms
        if not isinstance(x, type(object))]

## Including other syntaxes

[`import deathbeds.__Custom_display_formatting`](2018-09-03-Custom-display-formatting.ipynb) supports more syntaxes than Markdown.  We will include syntaxes for displaying images, webpages, and graphviz in a document.  If any 
of the values in `triggers` is found then we use Custom Display Formatters instead.  _These custom displays must be wrapped in strings when used in indented code blocks._

In [7]:
def fallback(ip, type, Exception, tb, **kwargs): 
    custom_exceptions = groupby(lambda x: issubclass(x, SyntaxError), ip.custom_exceptions)
    try:
        ip.custom_exceptions = tuple(x for x in custom_exceptions.get(False, []))
        if any(istrigger()):
            quote = '"""'
            if quote in ip.user_ns['In'][-1]: quote = "'''"
            ip.run_cell(quote+ip.user_ns['In'][-1]+quote)
        else:
            with swap_input_transformer(Literate()): 
                ip.run_cell(ip.user_ns['In'][-1])
    finally: ip.custom_exceptions = tuple(sum(custom_exceptions.values(), list()))
        
istrigger = lambda str=None: list(f(str or ip.user_ns['In'][-1]) for f in triggers)

### Create the extensions.

In [8]:
def load_ipython_extension(ip): 
    ip.set_custom_exc((SyntaxError,), fallback)
    %reload_ext deathbeds.__Custom_display_formatting
def unload_ipython_extension(ip): 
    %unload_ext deathbeds.__Custom_display_formatting
    ip.custom_exceptions = tuple(set(ip.custom_exceptions)-{SyntaxError})
    
if __name__ == '__main__': load_ipython_extension(get_ipython())

### tests

The tests should test different inputs and assure errors are being handled predictably.